# BENGALURU-HOUSE-PRICES
#### Võ Hửu Nghĩa

 ## I. DATA PROCESSING
 


 IMPORT CÁC THƯ VIỆN CẦN THIẾT

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

ĐỌC TẬP DỮ LIỆU BẰNG THƯ VIỆN PANDAS

In [ ]:
df1 = pd.read_csv('D:/Python-for-Data-Analyst/DataSet/bengaluru_house_prices.csv')
df1.head()

XEM SỐ DÒNG VÀ SỐ CỘT 

In [ ]:
df1.shape

- Dataset có 13320 dòng và 9 cột

XEM CHI TIẾT TỪNG CỘT VÀ KIỂU DỮ LIỆU CỦA CHÚNG

In [ ]:
df1.info()

ĐẾM GIÁ TRỊ NULL CỦA TỪNG CỘT

In [ ]:
df1.isnull().sum()

- Có 2 cột có số giá trị null cao là society(tên dự án) và balcony(số ban công có sẵn) với các giá trị lần lượt là : 5502 và 609
- Ngoài ra cố một số các cột khác cũng có giá trị null nhưng ít mà có thể loại bỏ hàng những đó được

XEM CÁC GIÁ TRỊ CÓ TRONG CỘT AREA_TYPE

In [ ]:
df1['area_type'].unique()

- Có 4 giá trị trong cột area_type

In [ ]:
df1['area_type'].value_counts()

- Ứng với mỗi giá trị  sẽ đi kèm số lượng ( tương ứng với số dòng xuất hiện trong tập dữ liệu)

XOÁ NHỮNG CỘT KHÔNG DÙNG ĐẾN TRONG VIỆC XÂY DỰNG MODEL

In [ ]:
 # tham số exis = columns để chỉ cho thằng pandas biết rằng những giá trị đó nằm ở trên cột nếu không để có thể sẽ xoá nhầm các hàng có giá trị đó.
df2 = df1.drop(['area_type','society','availability','balcony'],axis='columns')
df2.shape

## DATA CLEANING : XỬ LÍ GIÁ TRỊ NAN

KIỂM TRA CÁC GIÁ TRỊ NULL CÓ TRONG DF2

In [ ]:
df2.isnull().sum()

CHÚNG TA CÓ THỂ BỎ TẤT CẢ CÁC DÒNG CÓ GIÁ TRỊ NULL ĐÓ ĐI( NHƯNG SẼ KHÔNG HIỆU QUẢ ĐỐI VỚI CÁC GIÁ TRỊ CÓ QUÁ NHIỀU NULL, VÌ NHƯ THÊ SẼ LÀM MẤT ĐI TÀI NGUYÊN). NHƯNG ĐÂY LÀ CÁCH ĐƠN GIẢN NHẤT

In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

- Bây giờ đã không còn giá trụ null nào trong tập dữ liệu

In [ ]:
df3.shape

- Tập dữ liệu từ 13320 dòng (df2) xuống còn 13246 dòng (df3) nghĩa là đã loại bỏ đi 74 dòng chứa giá trị null

## FEATURE ENGINEERING

XEM LẠI TẬP DF3

In [ ]:
df3.sample(5)

- Trong cột size, có cả chữ và số, bây giờ t sẽ tách chúng ra làm 2 cột

tách size ra thành 2 cột 

In [ ]:
 # df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
# df3['bhk'] = df3['size'].str.extract(r'(\d+)', expand=False).astype(int)
df3.loc[:, 'bhk'] = df3['size'].str.extract(r'(\d+)', expand=False).astype(int)
df3.sample(4)

- tạo ra một cột mới là bhk để lưu giá trị đầu tiên của cột size, nghĩa là cột này chứa giá trị của số phòng.

Xoá những giá trị không phải là số trong trong cột total_sqft(chứa từ 1 chữ cái trở lên)

In [ ]:
# def remove_non_numeric(value):
#     return ''.join(char for char in value if char.isdigit())

# df3.loc[:, 'total_sqft'] = df3['total_sqft'].apply(remove_non_numeric)

Bây giờ xử lí cột total_sqft, vì cột này có các giá trị dạng 'số-số' ví dụ như 2100 - 2850, tôi sẽ tính trung bình của 2 số đấy, còn những giá trị khác thì vẫn giữa nguyên

In [ ]:
def calculate_average(value):
    if isinstance(value, str) and '-' in value:
        # If the value is in the format of number - number
        values = value.split('-')
        start = float(values[0].strip())
        end = float(values[1].strip())
        average = (start + end) / 2
        return average
    else:
        # If the value is a single number or a float
        try:
            return float(value)
        except ValueError:
            return None

df3['total_sqft'] = df3['total_sqft'].map(calculate_average)
df3.sample(2)

- Ở đây chúng ta ghi đè chứ không tạo ra cột mới để tiết kiệm dung lượng.

KIỂM TRA KẾT QUẢ ĐÃ XỬ LÍ Ở TRÊN

In [ ]:
df3.loc[30]

In [ ]:
(2100+2850)/2

- Kết quả cho ra hoàn toàn chính xác.

Tạo cột mới chứa giá của mỗi square feet. Giả sử giá 100000/sqf

In [ ]:
df3.loc[:, 'price_per_sqft'] = (df3['price']*100000/df3['total_sqft']).round(2)
df3.head(5)

- Ở đây chúng ta sử dụng phương thức loc() để tính toán trên tất cả các hàng ( bằng dấu :) và tạo ra cột mới chứa giá trị trên mỗi feet vuông
- Sau đó ta làm tròn 2 chữ số bằng hàm round(2)

SỬ DỤNG KĨ THUẬT GIẢM CHIỀU(DIMENSIONALITY REDUCTION) ĐỂ LÀM SỐ LƯỢNG LOCATION 

In [ ]:
# Xử dụng hàm hàm lambda với phương trức strip() nhầm loại bỏ những khoảng trắng ở đầu và cuối chuỗi
#Ví dụ, nếu giá trị ban đầu trong cột "location" là " New York ", sau khi áp dụng hàm lambda trên, giá trị sẽ trở thành "New York"
df3.location = df3.location.apply(lambda x: x.strip())
# Đếm số lượng của từng giá trị có trong cột localtion
location_status = df3['location'].value_counts(ascending=False)
location_status

TÍNH TỔNG TẤT CẢ CÁC GIÁ TRỊ TRÊN

In [ ]:
location_status.values.sum()


ĐẾM SỐ LƯỢNG PHẦN TỬ CÓ TRONG LOCATION_STATUS MÀ CÓ GIÁ TRỊ LỚN HƠN 10

In [ ]:
len(location_status[location_status > 10])


ĐẾM SỐ LƯỢNG PHẦN TỬ CÓ TRONG LOCATION_STATUS MÀ CÓ GIÁ TRỊ NHỎ HƠN HOẶC BẰNG 10

In [ ]:
len(location_status[location_status <= 10])

GIẢM CHIỀU DỮ LIỆU

In [ ]:
location_status_less_than_10 = location_status[location_status<=10]
location_status_less_than_10

THAY THẾ CÁC GIÁ TRỊ TRÊN BẰNG 'ORDER' CÒN CÁC GIÁ TRỊ KHÁC THÌ GIỮ NGUYÊN

In [ ]:
df3.location = df3.location.apply(lambda x: 'other' if x in location_status_less_than_10 else x)
len(df3.location.unique())

In [ ]:
df3.sample(10)

- Như ta thấy, một số location đã được chuyển thành other 

LOẠI BỎ OUTLIER SỬ DỤNG BUSINESS LOGIC

Thông thường mỗi phòng có diện tích là 300 feet vuông, nên ta sẽ loại bỏ những điểm dữ liệu mà nhỏ hơn số này là outlier

In [ ]:
df3[(df3['total_sqft']/df3['bhk'])<300].shape

- Sẽ có 744 hàng không đạt diện tích tối thiểu, và chúng sẽ được loại bỏ khỏi tập dữ liệu df3

In [ ]:
df4 = df3[~(df3['total_sqft']/df3['bhk']<300)]
df4.shape

- Lấy giá trị đảo của câu lệnh trên bằng dấu ngã '~' 
- Như vậy còn 12502 dòng sẽ được giữ lại để tiếp tục phân tích 

LOẠI BỎ OUTLIER BẰNG ĐỘ LỆCH CHUẨN VÀ GIÁ TRỊ TRUNG BÌNH

In [ ]:
a= df4.price_per_sqf